In [1]:
import random
import itertools
from multiprocessing import Pool

import matplotlib
import numpy as np
import scipy.linalg as la

%matplotlib inline

In [2]:
a = np.arange(1 * 3 * 3).reshape((1, 3, 3))
b = np.arange(1 * 3 * 3).reshape((1, 3, 3))
np.matmul(a,b).shape

(1, 3, 3)

In [3]:
A = np.array([[1,2,3],[3,7,7],[0,5,1]], dtype=np.uint8)
B = np.array([[1,2,5],[3,12,4],[0,5,0]], dtype=np.uint8)

In [4]:
A @ B

array([[  7,  41,  13],
       [ 24, 125,  43],
       [ 15,  65,  20]], dtype=uint8)

In [5]:
(A == B).all()

False

In [6]:
np.random.randint(3, size=(3, 3), dtype=np.uint8)

array([[1, 0, 0],
       [1, 0, 2],
       [0, 0, 1]], dtype=uint8)

In [7]:
np.array([[1,2,3],[3,7,7],[0,5,1]], dtype=np.uint8).reshape(-1)

array([1, 2, 3, 3, 7, 7, 0, 5, 1], dtype=uint8)

In [8]:
hash(tuple(np.array([[1,2,3],[3,7,7],[0,5,1]], dtype=np.uint8).reshape(-1)))

-2518969009246642181

In [9]:
pow(3, 9)

19683

In [10]:
class Matrix:
    P = 3
    
    @classmethod
    def rand(cls):
        m = np.random.randint(Matrix.P, size=(Matrix.P, Matrix.P), dtype=np.uint8)
        return cls(m=m)
        
    def __init__(self, idx=None, m=None):
        if m is not None:
            self.m = m
        elif idx is not None:
            assert 0 <= idx < 19683
            self.m = np.empty(shape=(Matrix.P, Matrix.P), dtype=np.uint8)

            v = idx
            for r in range(Matrix.P):
                for c in range(Matrix.P):
                    self.m[r][c] = v % Matrix.P
                    v = v // Matrix.P
                
    def idx(self):
        v = 0
        for r in reversed(range(Matrix.P)):
            for c in reversed(range(Matrix.P)):
                v = v * 3
                v += self.m[r][c]
        return v
        
    def __mul__(self, other):
        new_m = self.m @ other.m % Matrix.P
        return Matrix(m=new_m)
        
    def __repr__(self):
        return repr(self.m)
    
    def __str__(self):
        return str(self.m)
    
    def __hash__(self):
        return hash(tuple(self.m.reshape(-1)))
    
    def __eq__(self, other):
        return (self.m == other.m).all()
    
print(Matrix(0), Matrix(0).idx())
print(Matrix(1), Matrix(1).idx())

print('#2 * #2')
t = Matrix(2)
print(t, t.idx())
t = t * t
print(t, t.idx())

print(Matrix(19683 - 1), Matrix(19683 - 1).idx())

t = Matrix(19683 - 1) * Matrix(1)
print(t, t.idx())

[[0 0 0]
 [0 0 0]
 [0 0 0]] 0
[[1 0 0]
 [0 0 0]
 [0 0 0]] 1
#2 * #2
[[2 0 0]
 [0 0 0]
 [0 0 0]] 2
[[1 0 0]
 [0 0 0]
 [0 0 0]] 1
[[2 2 2]
 [2 2 2]
 [2 2 2]] 19682
[[2 0 0]
 [2 0 0]
 [2 0 0]] 1514


In [11]:
def one_step(s):

    def mat_mul(i):
        return i[0] * i[1]

    products = itertools.product(s, s)
    new_s = map(mat_mul, products)
    return s.union(new_s)

A = Matrix(idx = 2)
B = Matrix(idx = 5)
C = Matrix(idx = 100)
result = one_step({A, B, C})

sorted(list(map(lambda x:x.idx(), result)))

[1, 2, 5, 7, 11, 14, 100]

In [12]:
pool = Pool(32)

def mat_mul_by_idx(i):
    a = Matrix(idx = i[0])
    b = Matrix(idx = i[1])
    return (a * b).idx()

def one_step_by_idx(s):
    products = itertools.product(s, s)
    new_s = map(mat_mul_by_idx, products)
    return s.union(new_s)

# Cross-validate the multiplication results 
sorted(one_step_by_idx({2, 5, 100}))

[1, 2, 5, 7, 11, 14, 100]

In [13]:
def floodfill(s):
    last = s
    for i in range(100000):
        print(f'step #{i}, {len(last)}')
        new = one_step(last)
        if new == last:
            print(f'done, {len(last)}')
            break
        last = new
    return set(map(lambda x:x.idx(), last))

In [14]:
def floodfill_by_idx(s):
    last = s
    for i in range(100000):
        print(f'step #{i}, {len(last)}')
        new = one_step_by_idx(last)
        if new == last:
            print(f'done, {len(last)}')
            break
        last = new
    return last

In [15]:
def print_table(s):
    LINE_WIDTH = pow(3, 4)
    for i in range(pow(3, 9) // LINE_WIDTH):
        l = ''.join(['1' if (i*LINE_WIDTH + j) in s else '0' for j in range(LINE_WIDTH)])
        print(l)

In [17]:
A = Matrix(idx = 2)
print(A)
B = Matrix(idx = 100)
print(B)
C = Matrix(idx = 1001)
print(C)
s = {A, B, C}

result = floodfill(s)
print(result)
print()
print_table(result)

[[2 0 0]
 [0 0 0]
 [0 0 0]]
[[1 0 2]
 [0 1 0]
 [0 0 0]]
[[2 0 0]
 [1 0 1]
 [1 0 0]]
step #0, 3
step #1, 9
step #2, 19
done, 19
{1, 2, 11, 271, 8335, 19, 532, 28, 13862, 14375, 1459, 308, 56, 731, 100, 8038, 1513, 1001, 758}

011000000001000000010000000010000000000000000000000000001000000000000000000000000
000000000000000000010000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000010000000000000000000000000000000000001000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000010000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
000000000000000000000000000000000000000000000000000000000000000000000000000000000
0010000000000000000000000000010000000

In [ ]:
s = {1, 5, 10, 21, 50, 100, 600, 1000, 1999, 2001, 4001, 5000, 6000, 7000}
result = floodfill_by_idx(s)
print(result)
print()
print_table(result)

step #0, 14
step #1, 161
step #2, 2439
